# Analyses
- all neuronal pairwise correlations during sleep deprivation go here

# Distribution of pairwise correlations during SD from first to last hour of SD
- also compare with nsd 1st hour of sleep vs 5th hour sleep

In [ ]:
import numpy as np
import subjects
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from neuropy.plotting import Fig

sessions_sd = subjects.sd.allsess
sessions_nsd = subjects.nsd.allsess[:-1]


In [ ]:
%matplotlib inline
figure = Fig()
fig,gs = figure.draw(grid=(2,2),style='Pres')
gs_ = figure.subplot2grid(gs[:,1],grid=(5,2),hspace=0.5)
all_corr_sd = []

cdf = lambda x: np.cumsum(x)/np.sum(x)

bins = np.arange(-0.1,0.4,0.01)
for sub, sess in enumerate(sessions_sd):
    post = sess.paradigm['post']
    sd = [post[0],post[0]+5*3600]

    neurons = sess.neurons.get_neuron_type('pyr')
    cal_pair_corr=lambda t:neurons.get_binned_spiketrains(t_start=t,t_stop=t+3600).get_pairwise_corr()
    corr_pair = np.asarray( [cal_pair_corr(_) for _ in [sd[0],sd[0]+4*3600]])
    f_h = np.histogram(corr_pair[0],bins=bins)[0]
    l_h = np.histogram(corr_pair[1],bins=bins)[0]

    ax = plt.subplot(gs_[sub])
    ax.plot(bins[:-1],cdf(f_h))
    ax.plot(bins[:-1],cdf(l_h))
    ax.set_title(f'sub {sub}')

    all_corr_sd.append(corr_pair)

all_corr_nsd=[]
for sub, sess in enumerate(sessions_nsd,sub+2):
    post = sess.paradigm['post']

    neurons = sess.neurons.get_neuron_type('pyr')
    cal_pair_corr=lambda t:neurons.get_binned_spiketrains(t_start=t,t_stop=t+3600).get_pairwise_corr()
    corr_pair = np.asarray( [cal_pair_corr(_) for _ in [post[0],post[0]+4*3600]])
    f_h = np.histogram(corr_pair[0],bins=bins)[0]
    l_h = np.histogram(corr_pair[1],bins=bins)[0]

    ax = plt.subplot(gs_[sub])
    ax.plot(bins[:-1],cdf(f_h))
    ax.plot(bins[:-1],cdf(l_h))
    ax.set_title(f'sub {sub}')

    all_corr_nsd.append(corr_pair)




ax = plt.subplot(gs[0])
all_corr_sd = np.hstack(all_corr_sd)
first_h_sd = np.histogram(all_corr_sd[0],bins=bins)[0]
last_h_sd = np.histogram(all_corr_sd[1],bins=bins)[0]

all_corr_nsd = np.hstack(all_corr_nsd)
first_h_nsd = np.histogram(all_corr_nsd[0],bins=bins)[0]
last_h_nsd = np.histogram(all_corr_nsd[1],bins=bins)[0]

ax.plot(bins[:-1],cdf(first_h_sd),'r',label='sd_ZT1')
ax.plot(bins[:-1],cdf(last_h_sd),'g',label='sd_ZT5')
ax.plot(bins[:-1],cdf(first_h_nsd),'k',label='nsd_ZT1')
ax.plot(bins[:-1],cdf(last_h_nsd),'gray',label='nsd_ZT5')

ax.legend()
ax.set_xlabel('Pairwise correlation')
ax.set_ylabel('Probability')

# Pairwise correlation across time each session individually

In [ ]:
import subjects

In [ ]:
figure = Fig()
fig, gs = figure.draw(num=1, grid=(3, 2))
# sessions = subjects.Sd().allsess + subjects.Nsd().allsess
sessions = subjects.Sd().ratJday1

for sub, sess in enumerate(sessions):

    pre = sess.epochs.pre
    maze1 = sess.epochs.maze
    if maze1 is None:
        maze1 = sess.epochs.maze1
    post = sess.epochs.post

    # --- break region into periods --------
    bins = sess.utils.getinterval([post[0], post[0] + 5 * 3600], 2)
    sess.spikes.stability.firingRate(periods=bins)

    sess.replay.corr.across_time_window(period=[post[0], post[0] + 5 * 3600])
    ax = plt.subplot(gs[sub])
    sess.replay.corr.plot_across_time(ax=ax, smooth=1, tstart=maze1[0])

    corr_mat = sess.replay.corr.corr


# figure.savefig("RatS_EV", __file__)